# Let's go PRO!

Advanced RAG Techniques!

Let's start by digging into ingest:

1. No LangChain! Just native for maximum flexibility
2. Let's use an LLM to divide up chunks in a sensible way
3. Let's use the best chunk size and encoder from yesterday
4. Let's also have the LLM rewrite chunks in a way that's most useful ("document pre-processing")

In [166]:
from pathlib import Path
from groq import Groq          # 🔁 changed
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from chromadb import PersistentClient
from tqdm import tqdm
from litellm import completion
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go


load_dotenv(override=True)

MODEL = "groq/openai/gpt-oss-120b"      # 🔁 Groq model

DB_NAME = "preprocessed_db"
collection_name = "docs"
embedding_model = "text-embedding-3-large"
KNOWLEDGE_BASE_PATH = Path("knowledge-base")
AVERAGE_CHUNK_SIZE = 500

groq = Groq()                 # 🔁 Groq client




In [167]:
# Inspired by LangChain's Document - let's have something similar

class Result(BaseModel):
    page_content: str
    metadata: dict

In [168]:
# A class to perfectly represent a chunk

class Chunk(BaseModel):
    headline: str = Field(description="A brief heading for this chunk, typically a few words, that is most likely to be surfaced in a query")
    summary: str = Field(description="A few sentences summarizing the content of this chunk to answer common questions")
    original_text: str = Field(description="The original text of this chunk from the provided document, exactly as is, not changed in any way")

    def as_result(self, document):
        metadata = {"source": document["source"], "type": document["type"]}
        return Result(page_content=self.headline + "\n\n" + self.summary + "\n\n" + self.original_text,metadata=metadata)


class Chunks(BaseModel):
    chunks: list[Chunk]

## Three steps:

1. Fetch documents from the knowledge base, like LangChain did
2. Call an LLM to turn documents into Chunks
3. Store the Chunks in Chroma

### Let's start with Step 1

In [169]:
def fetch_documents():
    """A homemade version of the LangChain DirectoryLoader"""

    documents = []

    for folder in KNOWLEDGE_BASE_PATH.iterdir():
        doc_type = folder.name
        for file in folder.rglob("*.md"):
            with open(file, "r", encoding="utf-8") as f:
                documents.append({"type": doc_type, "source": file.as_posix(), "text": f.read()})

    print(f"Loaded {len(documents)} documents")
    return documents

In [170]:
documents = fetch_documents()

Loaded 76 documents


### On to Step 2 - make the chunks

In [171]:
# def make_prompt(document):
#     how_many = (len(document["text"]) // AVERAGE_CHUNK_SIZE) + 1
#     return f"""
# You take a document and you split the document into overlapping chunks for a KnowledgeBase.

# The document is from the shared drive of a company called Insurellm.
# The document is of type: {document["type"]}
# The document has been retrieved from: {document["source"]}

# A chatbot will use these chunks to answer questions about the company.
# You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
# This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
# There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

# For each chunk, you should provide a headline, a summary, and the original text of the chunk.
# Together your chunks should represent the entire document with overlap.

# Here is the document:

# {document["text"]}

# Respond with the chunks.
# """


def make_prompt(document):
    how_many = (len(document["text"]) // AVERAGE_CHUNK_SIZE) + 1
    return f"""
You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document["type"]}
The document has been retrieved from: {document["source"]}

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks.
This document should probably be split into {how_many} chunks.

Return a JSON object with a single key "chunks" containing the list of chunk objects.
Example: {{"chunks": [{{...}}, {{...}}]}}

Here is the document:

{document["text"]}
"""


In [172]:
print(make_prompt(documents[0]))


You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: contracts
The document has been retrieved from: knowledge-base/contracts/Contract with Belvedere Insurance for Markellm.md

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks.
This document should probably be split into 7 chunks.

Return a JSON object with a single key "chunks" containing the list of chunk objects.
Example: {"chunks": [{...}, {...}]}

Here is the document:

# Contract with Belvedere Insurance for Markellm

## Terms
This Contract ("Agreement") is made and entered into as of [Date] by and between Insurellm, Inc., a corporation registered in the United States, ("Provider") and Belvedere Insurance, ("Client"). 

1. **Service Commencement**: The services descr

In [173]:
def make_messages(document):
    return [
        {"role": "user", "content": make_prompt(document)},
    ]

In [174]:
make_messages(documents[0])

[{'role': 'user',
  'content': '\nYou take a document and you split the document into overlapping chunks for a KnowledgeBase.\n\nThe document is from the shared drive of a company called Insurellm.\nThe document is of type: contracts\nThe document has been retrieved from: knowledge-base/contracts/Contract with Belvedere Insurance for Markellm.md\n\nA chatbot will use these chunks to answer questions about the company.\nYou should divide up the document as you see fit, being sure that the entire document is returned in the chunks.\nThis document should probably be split into 7 chunks.\n\nReturn a JSON object with a single key "chunks" containing the list of chunk objects.\nExample: {"chunks": [{...}, {...}]}\n\nHere is the document:\n\n# Contract with Belvedere Insurance for Markellm\n\n## Terms\nThis Contract ("Agreement") is made and entered into as of [Date] by and between Insurellm, Inc., a corporation registered in the United States, ("Provider") and Belvedere Insurance, ("Client")

In [175]:

import os
GROQ_KEYS = os.getenv('GROQ_API_KEY2') 


In [176]:
# def process_document(document):
#     messages = make_messages(document)
#     response = completion(model=MODEL, messages=messages, response_format=Chunks)
#     reply = response.choices[0].message.content
#     doc_as_chunks = Chunks.model_validate_json(reply).chunks
#     return [chunk.as_result(document) for chunk in doc_as_chunks]


import time

def process_document(document):
    messages = make_messages(document)
    
    # NEW: Added num_retries to handle transient rate limits automatically
    response = completion(
        model=MODEL, 
        messages=messages, 
        response_format=Chunks,
        api_key=GROQ_KEYS,
        num_retries=3  # litellm will retry 3 times with backoff
    )
    
    reply = response.choices[0].message.content
    doc_as_chunks = Chunks.model_validate_json(reply).chunks
    return [chunk.as_result(document) for chunk in doc_as_chunks]

In [177]:
process_document(documents[0])

[Result(page_content='Contract Title and Parties\n\nIntroduces the contract between Insurellm and Belvedere Insurance, stating the parties involved and the effective date placeholder.\n\n# Contract with Belvedere Insurance for Markellm\n\n## Terms\nThis Contract ("Agreement") is made and entered into as of [Date] by and between Insurellm, Inc., a corporation registered in the United States, ("Provider") and Belvedere Insurance, ("Client"). ', metadata={'source': 'knowledge-base/contracts/Contract with Belvedere Insurance for Markellm.md', 'type': 'contracts'}),
 Result(page_content='Key Terms: Service, Duration, Fees, Payment\n\nDetails service start date, one‑year duration, fee structure ($199/month plus $25 per lead), and monthly payment schedule.\n\n1. **Service Commencement**: The services described herein will commence on [Start Date].\n2. **Contract Duration**: This Agreement shall remain in effect for a period of 1 year from the Commencement Date, unless terminated earlier in ac

In [182]:
# def create_chunks(documents):
#     chunks = []
#     for doc in tqdm(documents):
#         chunks.extend(process_document(doc))
#         # time.sleep(2)   # prevent token burst
#     return chunks

def create_chunks(documents):
    chunks = []
    for doc in tqdm(documents):
        chunks.extend(process_document(doc))
        
        # NEW: Mandatory sleep to avoid hitting the 8000 TPM limit
        # 15-20 seconds is safer for the Groq Free/On-demand tier
        time.sleep(20) 
        
    return chunks

In [183]:
chunks = create_chunks(documents)

 36%|███▌      | 27/76 [11:58<23:50, 29.20s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



 38%|███▊      | 29/76 [13:03<23:33, 30.08s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



 55%|█████▌    | 42/76 [18:17<13:31, 23.86s/it]

...

 97%|█████████▋| 74/76 [1:14:05<03:01, 90.90s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.I

 99%|█████████▊| 75/76 [1:15:55<01:36, 96.79s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.I

100%|██████████| 76/76 [1:17:46<00:00, 61.40s/it] 


In [184]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.


### Well that was easy! If a bit slow.

In the python module version, I sneakily use the multi-processing Pool to run this in parallel,
but if you get a Rate Limit Error you can turn this off in the code.

### Finally, Step 3 - save the embeddings

In [185]:
# def create_embeddings(chunks):
#     chroma = PersistentClient(path=DB_NAME)
#     if collection_name in [c.name for c in chroma.list_collections()]:
#         chroma.delete_collection(collection_name)

#     texts = [chunk.page_content for chunk in chunks]
#     emb = openai.embeddings.create(model=embedding_model, input=texts).data
#     vectors = [e.embedding for e in emb]

#     collection = chroma.get_or_create_collection(collection_name)

#     ids = [str(i) for i in range(len(chunks))]
#     metas = [chunk.metadata for chunk in chunks]

#     collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metas)
#     print(f"Vectorstore created with {collection.count()} documents")


from sentence_transformers import SentenceTransformer

# load model once (global)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def create_embeddings(chunks):
    chroma = PersistentClient(path=DB_NAME)
    if collection_name in [c.name for c in chroma.list_collections()]:
        chroma.delete_collection(collection_name)

    texts = [chunk.page_content for chunk in chunks]

    # create embeddings locally (no API)
    vectors = embedding_model.encode(texts, show_progress_bar=True).tolist()

    collection = chroma.get_or_create_collection(collection_name)

    ids = [str(i) for i in range(len(chunks))]
    metas = [chunk.metadata for chunk in chunks]

    collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metas)
    print(f"Vectorstore created with {collection.count()} documents")


In [186]:
create_embeddings(chunks)

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Vectorstore created with 272 documents


# Visual represnetation

In [187]:
chroma = PersistentClient(path=DB_NAME)
collection = chroma.get_or_create_collection(collection_name)
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [188]:
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [189]:
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

## And now - let's build an Advanced RAG!

We will use these techniques:

1. Reranking - reorder the rank results
2. Query re-writing

In [190]:
class RankOrder(BaseModel):
    order: list[int] = Field(
        description="The order of relevance of chunks, from most relevant to least relevant, by chunk id number"
    )

In [201]:
# def rerank(question, chunks):
#     system_prompt = """
# You are a document re-ranker.
# You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
# The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance, but you may be able to improve on that.
# You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.
# Reply only with the list of ranked chunk ids, nothing else. Include all the chunk ids you are provided with, reranked.
# """
#     user_prompt = f"The user has asked the following question:\n\n{question}\n\nOrder all the chunks of text by relevance to the question, from most relevant to least relevant. Include all the chunk ids you are provided with, reranked.\n\n"
#     user_prompt += "Here are the chunks:\n\n"
#     for index, chunk in enumerate(chunks):
#         user_prompt += f"# CHUNK ID: {index + 1}:\n\n{chunk.page_content}\n\n"
#     user_prompt += "Reply only with the list of ranked chunk ids, nothing else."
#     messages = [
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": user_prompt},
#     ]
#     response = completion(model=MODEL, messages=messages, response_format=RankOrder)
#     reply = response.choices[0].message.content
#     order = RankOrder.model_validate_json(reply).order
#     print(order)
#     return [chunks[i - 1] for i in order]

def rerank(question, chunks):
    system_prompt = """
You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance, but you may be able to improve on that.
You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.
Reply only with the list of ranked chunk ids, nothing else. Include all the chunk ids you are provided with, reranked.
"""

    user_prompt = f"The user has asked the following question:\n\n{question}\n\n"
    user_prompt += "Order all the chunks of text by relevance.\n\nHere are the chunks:\n\n"

    for index, chunk in enumerate(chunks):
        user_prompt += f"# CHUNK ID: {index + 1}:\n\n{chunk.page_content}\n\n"

    user_prompt += "Reply only with the list of ranked chunk ids, nothing else."

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    response = completion(model=MODEL, messages=messages, response_format=RankOrder)
    reply = response.choices[0].message.content

    order = RankOrder.model_validate_json(reply).order
    print("Model order:", order)

    # ✅ SAFETY FILTER (minimal fix)
    valid_ids = [i for i in order if 1 <= i <= len(chunks)]

    return [chunks[i - 1] for i in valid_ids]


In [202]:
# RETRIEVAL_K = 10

# def fetch_context_unranked(question):
#     query = openai.embeddings.create(model=embedding_model, input=[question]).data[0].embedding
#     results = collection.query(query_embeddings=[query], n_results=RETRIEVAL_K)
#     chunks = []
#     for result in zip(results["documents"][0], results["metadatas"][0]):
#         chunks.append(Result(page_content=result[0], metadata=result[1]))
#     return chunks

RETRIEVAL_K = 10

def fetch_context_unranked(question):
    # create query embedding locally
    query = embedding_model.encode(question).tolist()

    results = collection.query(query_embeddings=[query], n_results=RETRIEVAL_K)

    chunks = []
    for result in zip(results["documents"][0], results["metadatas"][0]):
        chunks.append(Result(page_content=result[0], metadata=result[1]))

    return chunks


In [203]:
question = "Who won the IIOTY award?"
chunks = fetch_context_unranked(question)

In [226]:
for chunk in chunks:
    print(chunk.page_content[:])

Additional HR Notes

Marcus holds a BA in Business Administration from NYU Stern and certifications including CCSM and Salesforce Administrator. He has earned the 2023 Customer Champion Award and actively mentors new associates, noted for strong relationship building and strategic advising.

## Other HR Notes
- **Education:** BA in Business Administration from NYU Stern School of Business
- **Certifications:** Certified Customer Success Manager (CCSM), Salesforce Administrator
- **Recognition:** Customer Champion Award 2023 for highest NPS scores
- **Mentorship:** Actively mentors new Customer Success Associates
- **Feedback:** Excellent relationship builder with deep product knowledge. Strong strategic advisor to clients. Known for going above and beyond to ensure client success.
Career Progression at Insurellm

From 2020 to 2023 Samantha advanced from HR Coordinator to a pivotal role in diversity and inclusion initiatives. Her responsibilities grew to include recruitment, employee re

In [205]:
reranked = rerank(question, chunks)

Model order: [23715468910]


In [206]:
for chunk in reranked:
    print(chunk.page_content[:15]+"...")

In [207]:
question = "Who went to Manchester University?"
RETRIEVAL_K = 20
chunks = fetch_context_unranked(question)
for index, c in enumerate(chunks):
    if "manchester" in c.page_content.lower():
        print(index)

In [208]:
reranked = rerank(question, chunks)

Model order: [1611018202345678911121314151719]


In [227]:
for index, c in enumerate(reranked):
    if "manchester" in c.page_content.lower():
        print(index)

In [213]:
if reranked:
    print(reranked[0].page_content)
else:
    print("Reranker failed, using top retrieved chunk instead:")
    print(chunks[0].page_content)


Reranker failed, using top retrieved chunk instead:
Additional HR Notes

Marcus holds a BA in Business Administration from NYU Stern and certifications including CCSM and Salesforce Administrator. He has earned the 2023 Customer Champion Award and actively mentors new associates, noted for strong relationship building and strategic advising.

## Other HR Notes
- **Education:** BA in Business Administration from NYU Stern School of Business
- **Certifications:** Certified Customer Success Manager (CCSM), Salesforce Administrator
- **Recognition:** Customer Champion Award 2023 for highest NPS scores
- **Mentorship:** Actively mentors new Customer Success Associates
- **Feedback:** Excellent relationship builder with deep product knowledge. Strong strategic advisor to clients. Known for going above and beyond to ensure client success.


In [214]:
def fetch_context(question):
    chunks = fetch_context_unranked(question)
    return rerank(question, chunks)

In [215]:
SYSTEM_PROMPT = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
Your answer will be evaluated for accuracy, relevance and completeness, so make sure it only answers the question and fully answers it.
If you don't know the answer, say so.
For context, here are specific extracts from the Knowledge Base that might be directly relevant to the user's question:
{context}

With this context, please answer the user's question. Be accurate, relevant and complete.
"""

In [216]:
# In the context, include the source of the chunk

def make_rag_messages(question, history, chunks):
    context = "\n\n".join(f"Extract from {chunk.metadata['source']}:\n{chunk.page_content}" for chunk in chunks)
    system_prompt = SYSTEM_PROMPT.format(context=context)
    return [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]

In [218]:
# def rewrite_query(question, history=[]):
#     """Rewrite the user's question to be a more specific question that is more likely to surface relevant content in the Knowledge Base."""
#     message = f"""
# You are in a conversation with a user, answering questions about the company Insurellm.
# You are about to look up information in a Knowledge Base to answer the user's question.

# This is the history of your conversation so far with the user:
# {history}

# And this is the user's current question:
# {question}

# Respond only with a single, refined question that you will use to search the Knowledge Base.
# It should be a VERY short specific question most likely to surface content. Focus on the question details.
# Don't mention the company name unless it's a general question about the company.
# IMPORTANT: Respond ONLY with the knowledgebase query, nothing else.
# """
#     response = completion(model=MODEL, messages=[{"role": "system", "content": message}])
#     return response.choices[0].message.content


def rewrite_query(question, history=[]):
    """Rewrite the user's question to be a more specific question that is more likely to surface relevant content in the Knowledge Base."""
    message = f"""
You are in a conversation with a user, answering questions about the company Insurellm.
You are about to look up information in a Knowledge Base to answer the user's question.

This is the history of your conversation so far with the user:
{history}

And this is the user's current question:
{question}

Respond only with a single, refined question that you will use to search the Knowledge Base.
It should be a VERY short specific question most likely to surface content. Focus on the question details.
Don't mention the company name unless it's a general question about the company.
IMPORTANT: Respond ONLY with the knowledgebase query, nothing else.
"""
    response = completion(model=MODEL, messages=[{"role": "system", "content": message}])
    return response.choices[0].message.content


In [219]:
rewrite_query("Who won the IIOTY award?", [])

'Who won the IIOTY award?'

In [231]:
def answer_question(question: str, history: list[dict] = []) -> tuple[str, list]:
    """
    Answer a question using RAG and return the answer and the retrieved context
    """
    query = rewrite_query(question, history)
    print(query)
    chunks = fetch_context(query)
    messages = make_rag_messages(question, history, chunks)
    response = completion(model=MODEL, messages=messages)
    return response.choices[0].message.content, chunks

In [232]:
answer_question("Who won the IIOTY award?", [])

Who won the IIOTY award?

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `openai/gpt-oss-120b` in organization `org_01kegx8yzce1fs824p93pwn7we` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 199649, Requested 4072. Please try again in 26m47.472s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


In [230]:
answer_question("Who went to Manchester University?", [])

InsureLLM employees who attended Manchester University

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `openai/gpt-oss-120b` in organization `org_01kegx8yzce1fs824p93pwn7we` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 199356, Requested 4586. Please try again in 28m22.944s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
